In [1]:
import tensorflow as tf
import utils
import numpy as np
import random as rn
import os
from tensorflow.keras.layers import Embedding

In [33]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

data = np.load(os.path.join('..','data','royalty.npz'))
RULE = 'aunt'



triples, traces = data[RULE + '_triples'], data[RULE + '_traces']
entities = data[RULE + '_entities'].tolist()
relations = data[RULE + '_relations'].tolist()  
ent2idx = dict(zip(entities, range(NUM_ENTITIES)))
rel2idx = dict(zip(relations, range(NUM_RELATIONS)))


NUM_ENTITIES = len(entities)
NUM_RELATIONS = len(relations)

In [3]:
#triples, traces = data['grandmother_triples'], data['grandmother_traces']

In [74]:
class RGCN_Layer(tf.keras.layers.Layer):
    def __init__(self,num_entities,num_relations,output_dim,seed,**kwargs):
        super(RGCN_Layer,self).__init__(**kwargs)
        self.num_entities = num_entities
        self.num_relations = num_relations
        self.output_dim = output_dim
        self.seed = seed
        
    def build(self,input_shape):

        input_dim = int(input_shape[-2][-1])
        
        self.relation_kernel = self.add_weight(
            shape=(self.num_relations,input_dim, self.output_dim),
            name="relation_kernels",
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=self.seed
            )
        )

        self.self_kernel = self.add_weight(
            shape=(input_dim, self.output_dim),
            name="self_kernel",
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=self.seed
            )
        )
    
    # def call(self, inputs):
        
    #     embeddings,head_idx,head_e,tail_idx,tail_e,adj_mats = inputs
            
    #     head_output = tf.matmul(head_e,self.self_kernel)
    #     tail_output = tf.matmul(tail_e,self.self_kernel)
                
    #     for i in range(self.num_relations):
            
    #         adj_i = adj_mats[i]
            
    #         head_adj = tf.nn.embedding_lookup(adj_i,head_idx)
    #         tail_adj = tf.nn.embedding_lookup(adj_i,tail_idx)
            
    #         head_update = tf.matmul(head_adj,embeddings)
    #         tail_update = tf.matmul(tail_adj,embeddings)

    #         head_output += tf.matmul(head_update,self.relation_kernel[i])
    #         tail_output += tf.matmul(tail_update,self.relation_kernel[i])
       
    #     return head_output, tail_output

    def call(self,inputs):

        embeddings,head_idx,head_e,tail_idx,tail_e,*adj_mats = inputs
                        
        head_output = tf.matmul(head_e,self.self_kernel)
        tail_output = tf.matmul(tail_e,self.self_kernel)
                
        for i in range(self.num_relations):
            
            adj_i = tf.sparse.reshape(adj_mats[0][i],shape=(self.num_entities,self.num_entities))

            sum_embeddings = tf.sparse.sparse_dense_matmul(adj_i, embeddings)
            
            head_update = tf.nn.embedding_lookup(sum_embeddings,head_idx)
            tail_update = tf.nn.embedding_lookup(sum_embeddings,tail_idx)
            
            head_output += tf.matmul(head_update,self.relation_kernel[i])
            tail_output += tf.matmul(tail_update,self.relation_kernel[i])
       
        return tf.sigmoid(head_output), tf.sigmoid(tail_output)

class DistMult(tf.keras.layers.Layer):
    def __init__(self, num_relations,seed,**kwargs):
        super(DistMult,self).__init__(**kwargs)
        self.num_relations = num_relations
        self.seed = seed
        
    def build(self,input_shape):
        
        embedding_dim = input_shape[0][-1]
        
        self.kernel = self.add_weight(
            shape=(self.num_relations,embedding_dim),
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=self.seed
            ),
            name='rel_embedding'
        )
        
    def call(self,inputs):
        
        head_e,head_idx,rel_idx,tail_e,tail_idx = inputs
        
        #head_e = tf.nn.embedding_lookup(head_e,head_idx)
        rel_e = tf.nn.embedding_lookup(self.kernel,rel_idx)
        
        score = tf.sigmoid(tf.reduce_sum(head_e*rel_e*tail_e,axis=-1))
        print(score)
        return tf.expand_dims(score,axis=0)

def RGCN_Model(num_entities,num_relations,embedding_dim,output_dim,seed):

    head_input = tf.keras.Input(shape=(None,), name='head_input',dtype=tf.int64)
    rel_input = tf.keras.Input(shape=(None,), name='rel_input',dtype=tf.int64)
    tail_input = tf.keras.Input(shape=(None,), name='tail_input',dtype=tf.int64)
    all_entities = tf.keras.Input(shape=(num_entities), name='all_entities',dtype=tf.int64)

    adj_inputs = tf.keras.Input(
        shape=(
            num_relations,
            num_entities,
            num_entities
        ),
        dtype=tf.float32,
        name='adj_inputs'
    )

    entity_embeddings = Embedding(
        input_dim=num_entities,
        output_dim=embedding_dim,
        name='entity_embeddings',
        embeddings_initializer=tf.keras.initializers.RandomUniform(
            minval=-1,
            maxval=1,
            seed=seed
        )
    )

    head_e = entity_embeddings(head_input)
    tail_e = entity_embeddings(tail_input)
    all_e = entity_embeddings(all_entities)

    new_head,new_tail = RGCN_Layer(num_relations=num_relations,output_dim=output_dim)([
        all_e,
        head_input,
        tail_input,
        head_e,
        tail_e,
        adj_inputs
        ]
    )

    output = DistMult(num_relations=num_relations,name='output')([
        new_head,rel_input,new_tail
        ]
    )

    model = tf.keras.Model(
        inputs=[
            all_entities,
            head_input,
            rel_input,
            tail_input,
            adj_inputs
        ],
        outputs=[
            output
        ]
    )

    return model

In [7]:
num_entities = NUM_ENTITIES
num_relations = NUM_RELATIONS
head_input = tf.keras.Input(shape=(None,), name='head_input',dtype=tf.int64)
rel_input = tf.keras.Input(shape=(None,), name='rel_input',dtype=tf.int64)
tail_input = tf.keras.Input(shape=(None,), name='tail_input',dtype=tf.int64)
all_entities = tf.keras.Input(shape=(num_entities), name='all_entities',dtype=tf.int64)

In [11]:
adj_inputs = [tf.keras.Input(
    shape=(num_entities,num_entities),
    dtype=tf.float32,
    name='adj_inputs_'+str(i),
    sparse=True,
    ) for i in range(num_relations)]

entity_embeddings = Embedding(
    input_dim=num_entities,
    output_dim=100,
    name='entity_embeddings',
    embeddings_initializer=tf.keras.initializers.RandomUniform(
        minval=-1,
        maxval=1,
        seed=SEED
    )
)


In [13]:
from tensorflow.keras.layers import Embedding, Lambda

head_e = entity_embeddings(head_input)
tail_e = entity_embeddings(tail_input)
all_e = entity_embeddings(all_entities)

head_e = Lambda(lambda x:x[0,:,:])(head_e)
tail_e = Lambda(lambda x:x[0,:,:])(tail_e)
all_e = Lambda(lambda x:x[0,:,:])(all_e)

head_index = Lambda(lambda x:x[0,:])(head_input)
rel_index = Lambda(lambda x:x[0,:])(rel_input)
tail_index = Lambda(lambda x:x[0,:])(tail_input)

In [75]:
new_head,new_tail = RGCN_Layer(
        num_relations=num_relations,
        num_entities=num_entities,
        output_dim=100,
        seed=SEED)([
            all_e,
            head_index,
            head_e,
            tail_index,
            tail_e,
            adj_inputs
            ]
        )
output = DistMult(num_relations=num_relations,seed=SEED,name='output')([
        new_head,head_index,rel_index,new_tail,tail_index
        ]
    )

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity t

In [58]:
train2idx = utils.array2idx(triples, ent2idx,rel2idx)
adj_mats = utils.get_adj_mats(train2idx,NUM_ENTITIES,NUM_RELATIONS)
train2idx = np.expand_dims(train2idx,axis=0)

In [41]:
all_indices = np.arange(NUM_ENTITIES).reshape(1,-1)

In [76]:
m = tf.keras.Model(inputs=[all_entities,head_input,rel_input,tail_input] + adj_inputs,
               outputs=[output])

In [77]:
m([all_indices,
            train2idx[:,:,0],
            train2idx[:,:,1],
            train2idx[:,:,2],
            adj_mats])

tf.Tensor([0.8569155  0.5661651  0.38363206 ... 0.00687283 0.6965849  0.7926308 ], shape=(7224,), dtype=float32)


<tf.Tensor: shape=(1, 7224), dtype=float32, numpy=
array([[0.8569155 , 0.5661651 , 0.38363206, ..., 0.00687283, 0.6965849 ,
        0.7926308 ]], dtype=float32)>

In [ ]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

data = np.load(os.path.join('..','data','royalty.npz'))

entities = data['entities'].tolist()
relations = data['relations'].tolist()

NUM_ENTITIES = len(entities)
NUM_RELATIONS = len(relations)
EMBEDDING_DIM = 3
OUTPUT_DIM = 5
LEARNING_RATE = 1e-3
NUM_EPOCHS = 1

ent2idx = dict(zip(entities, range(NUM_ENTITIES)))
rel2idx = dict(zip(relations, range(NUM_RELATIONS)))

triples, traces = data['spouse_triples'], data['spouse_traces']

train2idx = utils.array2idx(triples, ent2idx,rel2idx)[0:50]

# adj_mats = utils.get_adjacency_matrix_list(
#     num_relations=NUM_RELATIONS,
#     num_entities=NUM_ENTITIES,
#     data=train2idx
# )

train2idx = np.expand_dims(train2idx,axis=0)

all_indices = np.arange(NUM_ENTITIES).reshape(1,-1)

model = RGCN_Model(
    num_entities=NUM_ENTITIES,
    num_relations=NUM_RELATIONS,
    embedding_dim=EMBEDDING_DIM,
    output_dim=OUTPUT_DIM,
    seed=SEED
)

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
bce = tf.keras.losses.BinaryCrossentropy()

data = tf.data.Dataset.from_tensor_slices((
        train2idx[0,:,0],
        train2idx[0,:,1],
        train2idx[0,:,2], 
        np.ones(train2idx.shape[1])
    )
).batch(32)

for epoch in range(NUM_EPOCHS):

    for pos_head,rel,pos_tail,y in data:

        neg_head, neg_tail = utils.get_negative_triples(
            head=pos_head, 
            rel=rel, 
            tail=pos_tail,
            num_entities=NUM_ENTITIES
        )

        with tf.GradientTape() as tape:

            y_pos_pred = model([
                all_indices,
                pos_head,
                rel,
                pos_tail,
                adj_mats
                ],
                training=True
            )

            y_neg_pred = model([
                all_indices,
                neg_head,
                rel,
                neg_tail,
                adj_mats
                ],
                training=True
            )

            y_pred = tf.concat([y_pos_pred,y_neg_pred],axis=0)
            y_true = tf.concat([y,tf.zeros_like(y)],axis=0)

            loss = bce(y_true,y_pred)

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

    print(f'loss {loss} after epoch {epoch}')

In [ ]:
preds = model.predict(
    x=[
        all_indices,
        train2idx[:,:,0],
        train2idx[:,:,1],
        train2idx[:,:,2],
        adj_mats
    ]
)

In [ ]:
print(f'acc {(preds > .5).sum()/train2idx.shape[1]}')

In [ ]:
train2idx = utils.array2idx(triples, ent2idx,rel2idx)

adj_mats = []
for i in range(num_relations):
    
    adj = np.zeros((num_entities,num_entities))
    
    for h,_,t in (train2idx[train2idx[:,1] == i]):
        adj[h,t] = 1
        adj[t,h] = 1
        
    adj_mats.append(adj)
    
def neighbors(train2idx, entities):
    a_list = [[] for _ in entities]
    for h,r,t in train2idx:
        a_list[h].append([r, t])
        a_list[t].append([r, h])

    degrees = np.array([len(a) for a in a_list])
    a_list = [np.array(a) for a in a_list]
                 
    return a_list, degrees

a_list, degrees = neighbors(train2idx,entities)

train2idx = np.expand_dims(train2idx,axis=0)
adj_mats = np.expand_dims(adj_mats,0)

In [ ]:
# def get_neighbors(train_triples,entities):
    
#     adj_list = [[] for _ in entities]
#     for i,triplet in enumerate(train_triples):
#         adj_list[triplet[0]].append([i, triplet[2]])
#         #adj_list[triplet[2]].append([i, triplet[0]])

#     degrees = np.array([len(a) for a in adj_list])
#     adj_list = [np.array(a) for a in adj_list]

#     return adj_list, degrees

# adj_list, degrees = get_neighbors(train2idx,entities)

In [ ]:
for idx,i in enumerate(a_list):
    if i.shape[0] > 2:
        print(idx)
        break

In [ ]:
a_list[50]

In [ ]:
for idx,i in enumerate(train2idx[0]):
    
    h,r,t = i
    
    if h == 50:
        
        print(idx)

In [ ]:
adj_mats[0,2,:,:].sum(axis=0)[50]

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import RandomUniform
from tensorflow.python.ops import embedding_ops

entity_embeddings = Embedding(
    input_dim=num_entities,
    output_dim=embedding_dim,
    name='entity_embeddings',
    embeddings_initializer=RandomUniform(
        minval=-1,
        maxval=1,
        seed=SEED
        )
    )

In [ ]:
head_input = tf.keras.Input(shape=(), name='head_input',dtype=tf.int64)
rel_input = tf.keras.Input(shape=(), name='rel_input',dtype=tf.int64)
tail_input = tf.keras.Input(shape=(), name='tail_input',dtype=tf.int64)
all_entities = tf.keras.Input(shape=(num_entities), name='all_entities',dtype=tf.int64)

In [ ]:
#adj_inputs = [tf.keras.Input(shape=(num_relations,num_entities,num_entities),dtype=tf.float32,name='adj_'+str(i)) for i in range(num_relations)]

adj_inputs = tf.keras.Input(shape=(num_relations,num_entities,num_entities),dtype=tf.float32,name='adj_inputs')

In [ ]:
# print(all_e)
# print(head_input)
# print(tail_input)
# print(head_e)
# print(tail_e)
# print(adj_inputs)

In [ ]:
head_e = entity_embeddings(head_input)
tail_e = entity_embeddings(tail_input)
all_e = entity_embeddings(all_entities)


In [ ]:
#tf.matmul(tf.ones((1,5)), tf.ones((5,10)))
#tf.keras.backend.dot(tf.ones((10,1)),tf.ones((1,5)))
#tf.squeeze(tf.tensordot(tf.ones((5,5)),tf.ones((5,1)),axes=1),axis=1)
#tf.tensordot(tf.ones((5,5)),tf.ones((5,5)),axes=1)

In [ ]:
# class RGCN_Layer(tf.keras.layers.Layer):
#     def __init__(self,num_relations,output_dim,**kwargs):
#         super(RGCN_Layer,self).__init__(**kwargs)
#         self.num_relations = num_relations
#         self.output_dim = output_dim
    
#     def build(self,input_shape):

#         in_shape = input_shape[-1][-1]

#         self.W_r = self.add_weight(
#             shape=(self.num_relations,self.output_dim,in_shape),
#             trainable=True,
#             initializer="random_normal",
#             name='W_r'
#         )
        
#         self.W0 = self.add_weight(
#             shape=(self.output_dim,in_shape),
#             trainable=True,
#             initializer='random_normal',
#             name='W0'
#         )
        
#     def call(self,inputs):
        
#         head_input,tail_input = inputs
        
#         #filter_W_r = embedding_ops.embedding_lookup_v2(self.W_r,rel_input)
        
#         tail_update = tf.matmul(self.W_r,tail_input,transpose_b=True)
        
#         head_update = tf.matmul(self.W0,head_input,transpose_b=True)
        
#         update = tf.reduce_sum(tail_update + head_update, axis=0)

#         return tf.transpose(update)


# class RGCN_Layer(tf.keras.layers.Layer):
#     def __init__(self,num_relations,output_dim,**kwargs):
#         super(RGCN_Layer,self).__init__(**kwargs)
#         self.num_relations = num_relations
#         self.output_dim = output_dim
        
#     def build(self,input_shape):

#         #input_dim = int(input_shape[0][0])
#         input_dim = int(input_shape[2][-1])
        
#         self.relation_kernels = [
#                 self.add_weight(
#                     shape=(input_dim, self.output_dim),
#                     name="relation_kernels",
#                     trainable=True,
#                     initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED)
#                 )
#                 for _ in range(self.num_relations)
#             ]

#         self.self_kernel = self.add_weight(
#             shape=(input_dim, self.output_dim),
#             name="self_kernel",
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED)
#         )
        
#     def call(self, inputs):
        
#         head_idx,tail_idx,head_e, tail_e, *A_mats = inputs
 
#         head_output = tf.matmul(head_e,self.self_kernel)
#         tail_output = tf.matmul(tail_e,self.self_kernel)
        
#         for i in range(self.num_relations):
            
#             A = tf.squeeze(A_mats[i],axis=0)
#             #A = A_mats[i]

#             head_A_mat = embedding_ops.embedding_lookup_v2(A,head_idx)
#             tail_A_mat = embedding_ops.embedding_lookup_v2(A,tail_idx)
#             sliced_A_head = embedding_ops.embedding_lookup_v2(tf.transpose(head_A_mat),head_idx)
#             sliced_A_tail = embedding_ops.embedding_lookup_v2(tf.transpose(tail_A_mat),head_idx)
            
#             #print(sliced_A_head)

#             h_head = tf.tensordot(sliced_A_head, head_e,axes=1)
#             h_tail = tf.tensordot(sliced_A_tail, tail_e,axes=1)
            
#             head_output += tf.tensordot(h_head,self.relation_kernels[i],axes=1)
#             tail_output += tf.tensordot(h_tail,self.relation_kernels[i],axes=1)
        
#         return head_output,tail_output
        
#         for i in range(self.num_relations):
            
#             h = tf.tensordot(A_mats[i], features,axes=1)
#             output += tf.tensordot(h,self.relation_kernels[i],axes=1)
            
#         return tf.squeeze(output,axis=0)

# class RGCN_Layer(tf.keras.layers.Layer):
#     def __init__(self,num_relations,input_dim,output_dim,**kwargs):
#         super(RGCN_Layer,self).__init__(**kwargs)
#         self.num_relations = num_relations
#         self.input_dim = input_dim
#         self.output_dim = output_dim
        
#     def build(self,input_shape):
                
#         self.relation_kernels = self.add_weight(
#             shape=(self.num_relations,self.input_dim, self.output_dim),
#             name="relation_kernels",
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED)
#         )

#         self.self_kernel = self.add_weight(
#             shape=(self.input_dim, self.output_dim),
#             name="self_kernel",
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED)
#         )
        
#     def call(self,inputs):
        
#         adj_list, head_idx,rel_idx,tail_idx = inputs
        
#         #head_e = embedding_ops.embedding_lookup_v2(adj_list,head_idx)
#         head_e = adj_list[head_idx]
#         rel_e = 
                
#         return adj_list[head_idx]


#RGCN_Layer(num_relations=2,input_dim=3,output_dim=5)([adj_list,[6,7],[1,1],[2,2]])
#RGCN_Layer(num_relations=2,input_dim=3,output_dim=5)([adj_list,6,1,2])

#update custom training loop

# class RGCN_Layer(tf.keras.layers.Layer):
#     def __init__(self,num_relations,output_dim,**kwargs):
#         super(RGCN_Layer,self).__init__(**kwargs)
#         self.num_relations = num_relations
#         self.output_dim = output_dim
        
#     def build(self,input_shape):
        
#         input_dim = int(input_shape[0][-1])
        
#         self.relation_kernels = [
#                 self.add_weight(
#                     shape=(input_dim, self.output_dim),
#                     name="relation_kernels",
#                     trainable=True,
#                     initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED)
#                 )
#                 for _ in range(self.num_relations)
#             ]

#         self.self_kernel = self.add_weight(
#             shape=(input_dim, self.output_dim),
#             name="self_kernel",
#             trainable=True,
#             initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED)
#         )
        
#     def call(self, inputs):
        
#         features, *A_mats = inputs
#         print('features',features)
#         print('self_kernel',self.self_kernel)
#         #features = tf.squeeze(features, axis=0)
        
#         output = tf.matmul(features,self.self_kernel)
#         #print('output',output)
#         for i in range(self.num_relations):
            
#             #A_mat = tf.squeeze(A_mats[i],axis=0)
#             #print('A_mats',A_mats[i])
#             h = tf.tensordot(A_mats[i], features,axes=1)
#             #print('h',h)
#             output += tf.tensordot(h,self.relation_kernels[i],axes=1)
            
#         return output#tf.squeeze(output,axis=0)

class RGCN_Layer(tf.keras.layers.Layer):
    def __init__(self,num_relations,output_dim,**kwargs):
        super(RGCN_Layer,self).__init__(**kwargs)
        self.num_relations = num_relations
        self.output_dim = output_dim
        
    def build(self,input_shape):

        input_dim = int(input_shape[3][-1])
        
        self.relation_kernel = self.add_weight(
                    shape=(self.num_relations,input_dim, self.output_dim),
                    name="relation_kernels",
                    trainable=True,
                    initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED)
        )


        self.self_kernel = self.add_weight(
            shape=(input_dim, self.output_dim),
            name="self_kernel",
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED)
        )
    
    def call(self, inputs):
        
        embeddings,head_idx,tail_idx,head_e,tail_e,adj_mats = inputs
         
        head_output = tf.matmul(head_e,self.self_kernel)
        tail_output = tf.matmul(tail_e,self.self_kernel)
        
#         head_idx = tf.squeeze(head_idx,axis=0)
#         tail_idx = tf.squeeze(tail_idx,axis=0)        
        adj_mats = tf.squeeze(adj_mats,axis=0)
        embeddings = tf.squeeze(embeddings,axis=0)
        print('head_idx',head_idx)
        print('tail_idx',tail_idx)
        
        for i in range(self.num_relations):
            
            adj_i = adj_mats[i]

            head_adj = tf.nn.embedding_lookup(adj_i,head_idx)
            tail_adj = tf.nn.embedding_lookup(adj_i,tail_idx)
            
            h_head = tf.matmul(head_adj,embeddings)
            h_tail = tf.matmul(head_adj,embeddings)
            
            head_output += tf.matmul(h_head,self.relation_kernel[i])
            tail_output += tf.matmul(h_tail,self.relation_kernel[i])

        return head_output,tail_output
#         for i in range(self.num_relations):
            
#             adj_i = adj_mats[i]
            
#             head_adj = tf.transpose(tf.nn.embedding_lookup(adj_i,head_idx))
#             tail_adj = tf.transpose(tf.nn.embedding_lookup(adj_i,tail_idx))
            
#             h_head = tf.tensordot(head_adj, head_e,axes=1)
#             h_tail = tf.tensordot(tail_adj, tail_e,axes=1)

#             head_output += tf.tensordot(h_head,self.relation_kernel[i],axes=1)
#             tail_output += tf.tensordot(h_tail,self.relation_kernel[i],axes=1)  
            
#         head_output = tf.reduce_mean(head_output,axis=0)
#         tail_output = tf.reduce_mean(tail_output,axis=0)
        
#         return head_output,tail_output

#             h_head = tf.tensordot(head_adj, head_e,axes=1)
#             h_tail = tf.tensordot(tail_adj, tail_e,axes=1)
            
#             head_output += tf.tensordot(h_head,self.relation_kernel[i],axes=1)
#             tail_output += tf.tensordot(h_tail,self.relation_kernel[i],axes=1)  
            
#         return head_output,head_output


#         for i in range(self.num_relations):
            
#             A = tf.squeeze(A_mats[i],axis=0)
#             #A = A_mats[i]

#             head_A_mat = embedding_ops.embedding_lookup_v2(A,head_idx)
#             tail_A_mat = embedding_ops.embedding_lookup_v2(A,tail_idx)
#             sliced_A_head = embedding_ops.embedding_lookup_v2(tf.transpose(head_A_mat),head_idx)
#             sliced_A_tail = embedding_ops.embedding_lookup_v2(tf.transpose(tail_A_mat),head_idx)
            
#             #print(sliced_A_head)
# 
#             h_head = tf.tensordot(sliced_A_head, head_e,axes=1)
#             h_tail = tf.tensordot(sliced_A_tail, tail_e,axes=1)
            
#             head_output += tf.tensordot(h_head,self.relation_kernels[i],axes=1)
#             tail_output += tf.tensordot(h_tail,self.relation_kernels[i],axes=1)
        
#         return head_output,tail_output


In [ ]:
# RGCN_Layer(num_relations=num_relations,output_dim=5)([
#     entity_embeddings(np.arange(num_entities).reshape(1,-1)),
#     train2idx[0:,274:275,0],
#     train2idx[0:,274:275,2],
#     entity_embeddings(train2idx[0,274:275,0]),
#     entity_embeddings(train2idx[0,274:275,2]),
#     adj_mats
#     ]
# )
#entity_embeddings(np.arange(num_entities).reshape(1,-1))

In [ ]:
#RGCN_Layer(num_relations=num_relations,output_dim=5)([head_input,tail_input,head_e,tail_e] + adj_inputs)
#RGCN_Layer(num_relations=num_relations,output_dim=5)([entity_embeddings(tf.convert_to_tensor([1,2,3]))]+[np.eye(num_entities) for _ in relations])

# RGCN_Layer(num_relations=num_relations,output_dim=5)([
#     np.random.randn(num_entities,3),
#     train2idx[0:,274:275,0],
#     train2idx[0:,274:275,2],
#     entity_embeddings(train2idx[0,274:275,0]),
#     entity_embeddings(train2idx[0,274:275,2]),
#     adj_mats
#     ]
# )
RGCN_Layer(num_relations=num_relations,output_dim=5)([
    all_e,
    head_input,
    tail_input,
    head_e,
    tail_e,
    adj_inputs
    
])

#RGCN_Layer(num_relations=num_relations,output_dim=5)([head_input,tail_input,head_e,tail_e,adj_inputs])

#RGCN_Layer(num_relations=num_relations,output_dim=5)([np.random.randn(num_entities,5)]+[np.eye(num_entities) for _ in relations])

In [ ]:
class DistMult(tf.keras.layers.Layer):
    def __init__(self, num_relations,**kwargs):
        super(DistMult,self).__init__(**kwargs)
        self.num_relations = num_relations
        
    def build(self,input_shape):
        
        embedding_dim = input_shape[0][-1]
        
        self.kernel = self.add_weight(
            shape=(self.num_relations,embedding_dim),
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=1,seed=SEED),
            name='rel_embedding'
        )
        
    def call(self,inputs):
        
        head_e,rel_idx,tail_e = inputs
        
        rel_e = tf.nn.embedding_lookup(self.kernel,rel_idx)
        
        return tf.sigmoid(tf.reduce_sum(head_e*rel_e*tail_e, axis=-1))

In [ ]:
##### new_head = RGCN_Layer(num_relations=num_relations,output_dim=5)([head_e,tail_e])
# new_head = tf.keras.layers.Activation('sigmoid')(new_head)

# new_tail = RGCN_Layer(num_relations=num_relations,output_dim=5)([tail_e,head_e])
# new_tail = tf.keras.layers.Activation('sigmoid')(new_tail)

# output = DistMult(num_relations=num_relations)([new_head,rel_input,new_tail])
#DistMult(num_relations=num_relations)([np.ones((6,5)),[2,2],[2,2],[1,1]])
new_head,new_tail = RGCN_Layer(num_relations=num_relations,output_dim=5)([
    all_e,
    head_input,
    tail_input,
    head_e,
    tail_e,
    adj_inputs
    
])

output = DistMult(num_relations=num_relations,name='output')([new_head,rel_input,new_tail])

In [ ]:
#model = tf.keras.Model([head_input,rel_input,tail_input],[output])
output

In [ ]:
#model.compile(loss='binary_crossentropy', optimizer='sgd')

In [ ]:
#model.fit(x=[train2idx[:,0],train2idx[:,1],train2idx[:,2]],y=np.ones(train2idx.shape[0]),epochs=1)

In [ ]:
# class RGCN_Model(tf.keras.Model):
#     def __init__(self,num_entities,*args,**kwargs):
#         super(RGCN_Model,self).__init__(*args, **kwargs)
#         self.num_entities = num_entities
        
#     def train_step(self,data):

#         embeddings,pos_head,rel,pos_tail,*adj_mats = data[0]
#         y = data[1]
        
#         neg_head, neg_tail = utils.get_negative_triples(
#             head=pos_head, 
#             rel=rel, 
#             tail=pos_tail,
#             num_entities=self.num_entities
#             )
        
#         head = tf.concat([pos_head,neg_head],axis=0)
#         rel = tf.concat([rel,rel],axis=0)
#         tail = tf.concat([pos_tail,neg_tail],axis=0)
                
#         y = tf.concat([y,tf.zeros_like(y)],axis=0)
        
#         with tf.GradientTape() as tape:
            
#             y_pred = self([embeddings,head,rel,tail,*adj_mats],training=True)
            
#             loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
            
#         trainable_vars = self.trainable_variables
#         gradients = tape.gradient(loss, trainable_vars)

#         self.optimizer.apply_gradients(zip(gradients, trainable_vars))

#         self.compiled_metrics.update_state(y, y_pred)

#         return {m.name: m.result() for m in self.metrics}

In [ ]:
model = tf.keras.Model(
    inputs=[
        all_entities,
        head_input,
        rel_input,
        tail_input,
        adj_inputs
    ],
    outputs=[
        output
    ]
)

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
bce = tf.keras.losses.BinaryCrossentropy()
all_indices = np.arange(num_entities).reshape(1,-1)
# num_epochs = 1
# for epoch in range(num_epochs):
#     print(epoch)
data = tf.data.Dataset.from_tensor_slices((train2idx[0,:,0],train2idx[0,:,1],train2idx[0,:,2], np.ones(train2idx.shape[1]))).batch(32)

for pos_head,rel,pos_tail,y in data:
    
    neg_head, neg_tail = utils.get_negative_triples(
        head=pos_head, 
        rel=rel, 
        tail=pos_tail,
        num_entities=num_entities
    )
    
    with tf.GradientTape() as tape:
        
        y_pos_pred = model([
            all_indices,
            pos_head,
            rel,
            pos_tail,
            adj_mats
        ],
            training=True
        )
        
        y_neg_pred = model([
            all_indices,
            neg_head,
            rel,
            neg_tail,
            adj_mats
        ],
            training=True
        )
        
        y_pred = tf.concat([y_pos_pred,y_neg_pred],axis=0)
        y_true = tf.concat([y,tf.zeros_like(y)],axis=0)
        
        loss = bce(y_true,y_pred)

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    break


In [ ]:
# inputs = {**{"all_entities":np.arange(num_entities),
#      "head_input":train2idx[:,0],
#      "rel_input":train2idx[:,1],
#      "tail_input":train2idx[:,2]},**{"adj_"+str(i):adj_mats[i] for i in range(num_relations)}}

# model.inputs

preds_a = model.predict(
    x=[
        all_indices,
        train2idx[:,:,0],
        train2idx[:,:,1],
        train2idx[:,:,2],
        adj_mats
    ],
    batch_size=1
)
# preds_b = model.predict(
#     x=[
#         all_indices,
#         train2idx[:,:,0],
#         train2idx[:,:,1],
#         train2idx[:,:,2],
#         adj_mats
#     ]
# )

In [ ]:
#model.compile(loss='binary_crossentropy', optimizer='sgd')

In [ ]:
# model.fit(
#     x=[
#         train2idx[:,0],
#         train2idx[:,1],
#         train2idx[:,2],
#         *adj_mats
#     ],
#     y=np.ones(train2idx.shape[0]),
#     epochs=1,
#     verbose=0,
#     batch_size=1
# )

# model.fit(
#     inputs,
#     {"output":np.ones(train2idx.shape[0])},
#     epochs=1,
#     verbose=0,
#     batch_size=1
# )

In [ ]:
# import dgl
# import torch as th
# from dgl.nn import RelGraphConv
# g = dgl.graph(([0,1,2,3,2,5,0], [1,2,3,4,0,3,1]))
# conv = RelGraphConv(10, 5, 3, regularizer='basis')
# feat = th.ones(6, 10)
# etype = th.tensor(np.array([0,1,2,0,1,2,1]).astype(np.int64))
# conv(g, feat, etype)

In [ ]:
(model.predict([train2idx[:,0],train2idx[:,1],train2idx[:,2]]) > .5).sum()/train2idx.shape[0]

In [ ]:
#tf.sparse.sparse_dense_matmul(sparse_tensor, zeros)

In [ ]:
#zeros = tf.zeros(shape=[num_entities,num_relations,num_entities],dtype=tf.float64)
# adj = np.zeros(shape=[num_entities,num_relations,num_entities])

# for h,r,t in train2idx:
    
#     adj[h,r,t] = 1
    
# adj = tf.convert_to_tensor(adj)

In [ ]:

#get neighbors of head+tail
#update rgcn -> for each triple, feed in neighbors (update weight matrix?, normalize by degrees)
#GNNEXPLAINER: 

In [ ]:
#train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)

In [ ]:
# mask = tf.Variable(
#     initial_value=tf.random.uniform(
#         minval=0,
#         maxval=1,
#         shape=[
#             num_entities,
#             num_relations,
#             num_entities
#         ]
#     ),
#     trainable=True,
#     shape=[
#         num_entities,
#         num_relations,
#         num_entities
#     ],
#     name='mask'
# )


In [ ]:
i = 100

# model.predict(
#     x=[
#         train2idx[i,0].reshape(-1,1),
#         train2idx[i,1].reshape(-1,1),
#         train2idx[i,2].reshape(-1,1)
#     ],
#     batch_size=1
# )
model.predict(
    x=[
        train2idx[:,0],
        train2idx[:,1],
        train2idx[:,2]
    ]
)

In [ ]:
#for triple i:
    #get k hop subgraph?
    #compute pred 
    #define masks
    #optimize loss, return masks

In [ ]:
# def get_adjlist(train2idx,entities):

#     adj_list = [[] for _ in entities]

#     for i,triplet in enumerate(train2idx):
#         adj_list[triplet[0]].append([i, triplet[2]])
#         adj_list[triplet[2]].append([i, triplet[0]])

#     degrees = np.array([len(a) for a in adj_list])
#     adj_list = [np.array(a) for a in adj_list]
    
#     return adj_list,degrees

In [ ]:
#get_adjlist(train2idx,entities)